In [1]:
import os
import scipy.io
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from PIL import Image

# ==========================================
# 1. DATA ENGINEERING (Custom Dataset)
# ==========================================

class StanfordCarsDataset(Dataset):
    """
    Custom Dataset to handle Stanford Cars structure.
    Includes a 'Cleaning' step to remove images mentioned in the .mat file 
    that do not physically exist on the disk.
    """
    def __init__(self, root_dir, mat_file, image_folder, transform=None):
        self.root_dir = root_dir
        self.image_folder = image_folder
        self.transform = transform
        
        print(f"Loading annotations from: {mat_file}")
        
        # Load Matlab file
        self.annotations = scipy.io.loadmat(mat_file)
        raw_samples = self.annotations['annotations'][0]

        # --- NEW: FILTERING STEP ---
        # We iterate through the list and keep ONLY the images that actually exist.
        self.samples = []
        missing_count = 0
        
        print(f"Verifying {len(raw_samples)} images in dataset...")
        
        for sample in raw_samples:
            img_name = sample[-1][0] # Extract filename
            img_path = os.path.join(self.root_dir, self.image_folder, img_name)
            
            if os.path.exists(img_path):
                self.samples.append(sample)
            else:
                missing_count += 1
                # Print the first 3 missing files just for info
                if missing_count <= 3:
                    print(f"Warning: Skipped missing file: {img_name}")

        print(f"Done. {len(self.samples)} images valid. {missing_count} images missing/skipped.")
        
        # Map class IDs (1-196) to 0-195 for PyTorch
        self.classes = [str(i) for i in range(196)] 

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        ann = self.samples[idx]
        
        img_name = ann[-1][0] 
        label = ann[-2][0][0] - 1  

        img_path = os.path.join(self.root_dir, self.image_folder, img_name)
        
        # Now we can safely open, because we verified existence in __init__
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

# ==========================================
# 2. DATA PIPELINE SETUP
# ==========================================

def get_data_loaders(data_root, batch_size=32):
    """
    Creates Train and Validation loaders.
    Automatically searches for the .mat file in car_devkit/ folders.
    """
    
    # Standard ImageNet normalization
    stats = ((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    
    # 1. Define Transforms
    train_tfms = transforms.Compose([
        transforms.Resize((256, 256)),      
        transforms.RandomCrop(224),         
        transforms.RandomHorizontalFlip(),  
        transforms.ToTensor(),
        transforms.Normalize(*stats)
    ])

    valid_tfms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(*stats)
    ])

    # 2. Locate the Annotation File (Robust Check)
    # We check two common locations inside car_devkit
    possible_paths = [
        os.path.join(data_root, 'car_devkit', 'devkit', 'cars_train_annos.mat'),
        os.path.join(data_root, 'car_devkit', 'cars_train_annos.mat'),
        os.path.join(data_root, 'cars_train_annos.mat') # Fallback to root
    ]
    
    mat_path = None
    for path in possible_paths:
        if os.path.exists(path):
            mat_path = path
            break
            
    if mat_path is None:
        raise FileNotFoundError(
            f"Could not find 'cars_train_annos.mat'. \n"
            f"Checked inside: {data_root} and 'car_devkit' subfolders.\n"
            f"Please ensure your folder structure matches the code expectations."
        )

    # 3. Initialize Dataset
    full_dataset = StanfordCarsDataset(
        root_dir=data_root, 
        mat_file=mat_path,
        image_folder=r'cars_train/cars_train',  # <--- THIS IS THE FIX
        transform=train_tfms 
    )

    # 4. Split into Train (80%) and Validation (20%)
    # We use a stratified split to ensure all 196 classes are in both sets
    print("Splitting dataset into Train/Val...")
    labels = [full_dataset.samples[i][-2][0][0] for i in range(len(full_dataset))]
    
    train_indices, val_indices = train_test_split(
        range(len(full_dataset)), 
        test_size=0.2, 
        stratify=labels, 
        random_state=42
    )

    # Create Subsets
    train_set = torch.utils.data.Subset(full_dataset, train_indices)
    val_set = torch.utils.data.Subset(full_dataset, val_indices)
    
    # Overwrite transform for validation set (remove augmentation)
    val_set.dataset.transform = valid_tfms 

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0) # num_workers=0 is safer on Windows
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=0)

    return train_loader, val_loader

# ==========================================
# 3. RESNET MODEL ARCHITECTURE
# ==========================================

def get_resnet_model(num_classes=196):
    # Load Pre-trained ResNet50
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

    # --- STRATEGY: FINE-TUNING ---
    # 1. Freeze the early layers (generic features like lines/edges)
    for param in model.parameters():
        param.requires_grad = False
        
    # 2. UNFREEZE the last block (layer4)
    # This allows the model to learn specific "car" features
    for param in model.layer4.parameters():
        param.requires_grad = True
        
    # 3. Replace the Head
    in_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(in_features, 512),
        nn.ReLU(),
        nn.Dropout(0.3), # Helps prevent overfitting
        nn.Linear(512, num_classes)
    )

    return model
# ==========================================
# 4. TRAINING SKELETON
# ==========================================

if __name__ == "__main__":
    # --- CONFIGURATION ---
    # UPDATE THIS PATH to your actual folder path
    # Use 'r' before the string to handle Windows backslashes
    DATA_PATH = r"D:/deep_learning/stanford-cars-dataset" 
    
    # Check if path exists before starting
    if not os.path.exists(DATA_PATH):
        print(f" Error: The path '{DATA_PATH}' does not exist.")
        print("Please edit the DATA_PATH variable at the bottom of the script.")
    else:
        print(f" Data path found: {DATA_PATH}")
        print("Initializing Data Pipeline...")
        
        try:
            train_dl, val_dl = get_data_loaders(DATA_PATH)
            
            print("Initializing ResNet50...")
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = get_resnet_model(num_classes=196).to(device)
            
            # Hyperparameters
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=0.0001) 
            
            print(f"Starting Training on {device}...")
            num_epochs = 10
            
            for epoch in range(num_epochs):
                model.train()
                running_loss = 0.0
                correct_predictions = 0 
                total_samples = 0
                
                for images, labels in train_dl:
                    images, labels = images.to(device), labels.to(device)
                    
                    optimizer.zero_grad()
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    
                    running_loss += loss.item()
                    
                    # Track Accuracy during training to see progress
                    _, predicted = torch.max(outputs, 1)
                    total_samples += labels.size(0)
                    correct_predictions += (predicted == labels).sum().item()

                epoch_loss = running_loss / len(train_dl)
                epoch_acc = 100 * correct_predictions / total_samples
                
                print(f"Epoch {epoch+1}/{num_epochs} | Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.2f}%")
                
            print("Training loop finished successfully!")
            print("Saving model...")
            torch.save(model.state_dict(), "resnet50_stanford_cars.pth")
            print("Model saved as 'resnet50_stanford_cars.pth'")

        except Exception as e:
            print("\n An error occurred during execution:")
            print(e)
            # This helps debug if the .mat structure is slightly different
            import traceback
            traceback.print_exc()

 Data path found: D:/deep_learning/stanford-cars-dataset
Initializing Data Pipeline...
Loading annotations from: D:/deep_learning/stanford-cars-dataset\car_devkit\devkit\cars_train_annos.mat
Verifying 8144 images in dataset...
Done. 8144 images valid. 0 images missing/skipped.
Splitting dataset into Train/Val...
Initializing ResNet50...
Starting Training on cuda...
Epoch 1/10 | Loss: 4.9242 | Accuracy: 3.99%
Epoch 2/10 | Loss: 3.5960 | Accuracy: 19.14%
Epoch 3/10 | Loss: 2.3503 | Accuracy: 42.59%
Epoch 4/10 | Loss: 1.3754 | Accuracy: 65.48%
Epoch 5/10 | Loss: 0.7170 | Accuracy: 83.93%
Epoch 6/10 | Loss: 0.3706 | Accuracy: 93.32%
Epoch 7/10 | Loss: 0.2130 | Accuracy: 96.76%
Epoch 8/10 | Loss: 0.1197 | Accuracy: 98.50%
Epoch 9/10 | Loss: 0.0779 | Accuracy: 99.05%
Epoch 10/10 | Loss: 0.0578 | Accuracy: 99.39%
Training loop finished successfully!
Saving model...
Model saved as 'resnet50_stanford_cars.pth'
